In [1]:
#libraries
from geopy.geocoders import Yandex
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date as dt
from tqdm import tqdm
from pathlib import Path
import time
import warnings
warnings.filterwarnings("ignore")


In [2]:
#da eseguire solo la prima volta per installare la libreria
#!pip install pathlib

In [3]:
def find_streets(h):
    h = h[['Latitude', 'Longitude']]

    passed = []
    i = 0
    for i in range(len(h)):
        if (i % 100) == 0:
            print(i)
            
        try:
            location = geolocator.reverse((h.loc[i][0], h.loc[i][1]))

        except:
            print(i, 'WAIT')
            time.sleep(3)
            location = geolocator.reverse((h.loc[i][0], h.loc[i][1]))

        try:
                passed.append(location[0].raw['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['SubAdministrativeArea']['Locality']['DependentLocality']['DependentLocality']['Thoroughfare']['ThoroughfareName'])
        except:

            try:
                    passed.append(location[0].raw['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['SubAdministrativeArea']['Locality']['Thoroughfare']['ThoroughfareName'])
            except: 

                try:
                        passed.append(location[0].raw['metaDataProperty']['GeocoderMetaData']['AddressDetails']['Country']['AdministrativeArea']['SubAdministrativeArea']['Locality']['DependentLocality']['Thoroughfare']['ThoroughfareName'])

                except:
                        passed.append(None)
          

    return passed

In [4]:

geolocator = Yandex(user_agent="my-application", lang="en_US")
big_android = pd.DataFrame()
j = 706
pathlist = Path('').glob('**/*gps.csv')
for path in list(pathlist)[6:]:
    j += 1
    
    #import files
    file = str(path)
    print(file, j)
    h = pd.read_csv(file, skiprows=1) #gps

    try:
        r = pd.read_csv(r'./Android files/' + str(j) +'_sj.csv') #science_journal
    except FileNotFoundError as fnf_error:
        print(fnf_error)
        break

        
    #order/filter column
    col = []
    for i in h.columns:
        i = i.replace(' ', '')
        col.append(i)
    h.columns = col
    h['street'] = find_streets(h)

    h = h.filter(['Time','Latitude','Longitude','Elevation','Speed','street'])
    
    #to datetime
    h['Time'] = '2019-04-25 '+ h['Time']
    h['Time'] = pd.to_datetime(h.Time, format='%Y-%m-%d %H:%M:%S')
    
    #adjust Speed
    h['Speed'] = h.Speed*4
    
    #put n. person, type of car
    h['N_person'] = int(file.split('_')[2])
    h['Car_type'] = int(file.split('_')[3])
    
    #create diff
    data1=h.iloc[0]['Time']
    h['Diff']=(h['Time']-data1).dt.seconds
    h['Diff']=h['Diff']*1000
    h = h.drop_duplicates(subset=['Diff'], keep='first')
    
    #assign a reference to sj_file
    l=[str(i) for i in h.index]
    gap=[h.iloc[i]['Diff'] for i in range(len(h))]
    del l[-1]
    try:
        r=r.assign(tag=pd.cut(r['relative_time'],gap,labels=l))#,duplicates='drop'))
    except:
        r['relative_time'] = r['timestamp'].diff()
        r['relative_time'][0] = 0
        r.drop(['timestamp'], axis =1)
        r=r.assign(tag=pd.cut(r['relative_time'],gap,labels=l))#,duplicates='drop'))

    r['tag']=r['tag'].fillna(value='0')
    
    #adjust sj
    sj_new=r.filter(['relative_time','AccZ','tag'])
    sj_new.columns=['Time','Acc','Reference']
    sj_new['Reference'] = pd.to_numeric(sj_new['Reference'])
    
    #merged df
    df=pd.merge(sj_new,h,left_on='Reference',right_on=h.index)
    df=df.drop(columns=['Reference','Diff'])
    df.columns=['Elapse(ms)','Acc','Date','Latitude','Longitude','Elevation','Speed(Km/h)', 'Street','N_person','Car_type']
    df=df.reindex(['Date','Elapse(ms)','Latitude','Longitude', 'Street', 'Elevation','Speed(Km/h)','Acc','N_person','Car_type'],axis=1)
    df=df[df['Speed(Km/h)']>1]
    

    #append current df to previuos ones
    if len(big_android)==0:
        big_android = df
    else:
        big_android = big_android.append(df) 
        
print('hi')
big_android.to_csv('All_Records_Android'+ str(j)+ '.csv', index=False)

Android files\707_track_30_3_gps.csv 707
0
100
200
231 WAIT
300
400
500
600
700
800
897 WAIT
900
1000
1100
1169 WAIT
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2363 WAIT
2387 WAIT
2400
2437 WAIT
2500
2567 WAIT
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3565 WAIT
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4657 WAIT
4700
4800
4900
5000
5100
5200
5201 WAIT
5300
Android files\708_track_30_3_gps.csv 708
0
100
200
300
327 WAIT
400
500
600
700
800
900
962 WAIT
1000
1051 WAIT
1100
1200
1300
1400
1500
1600
1644 WAIT
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3385 WAIT
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
Android files\709_track_2_0_gps.csv 709
0
100
200
300
400
500
600
700
784 WAIT
800
900
1000
1100
Android files\710_track_2_0_gps.csv 710
0
26 WAIT
47 WAIT
50 WAIT
100
169 WAIT
200
300
400
500
522 WAIT
523 WAIT


GeocoderTimedOut: Service timed out

HOW TO SLICE THE LIST OF PATHS

In [6]:
pathlist = Path('').glob('**/*gps.csv')

list(pathlist)[6:]

[WindowsPath('Android files/707_track_30_3_gps.csv'),
 WindowsPath('Android files/708_track_30_3_gps.csv'),
 WindowsPath('Android files/709_track_2_0_gps.csv'),
 WindowsPath('Android files/710_track_2_0_gps.csv'),
 WindowsPath('Android files/711_track_30_3_gps.csv'),
 WindowsPath('Android files/712_track_30_3_gps.csv'),
 WindowsPath('Android files/713_track_3_1_gps.csv'),
 WindowsPath('Android files/714_track_3_1_gps.csv'),
 WindowsPath('Android files/715_track_2_0_gps.csv'),
 WindowsPath('Android files/716_track_2_0_gps.csv'),
 WindowsPath('Android files/717_track_2_0_gps.csv'),
 WindowsPath('Android files/718_track_4_0_gps.csv'),
 WindowsPath('Android files/719_track_2_0_gps.csv'),
 WindowsPath('Android files/720_track_2_0_gps.csv'),
 WindowsPath('Android files/721_track_2_0_gps.csv'),
 WindowsPath('Android files/722_track_2_0_gps.csv'),
 WindowsPath('Android files/723_track_3_0_gps.csv'),
 WindowsPath('Android files/724_track_2_0_gps.csv'),
 WindowsPath('Android files/725_track_2_0_

# SAVE THE FILES WITH THIS METHOD

In [5]:
big_android.to_csv('All_Records_Android_till_709.csv', index=False)